# Intro

This script builds a database of film metadata from a list of IMDB Film IDs supplied by the user.  It is structured as an ETL pipeline:

* The **Extract** stage takes the raw data from IMDB.  At the end of this stage, raw data is stored as a list-of-lists.
* The **Transform** stage normalizes the data from IMDB.  At the end of this stage, each table entry is complete and stored as a dictionary where the keys are column names. 
* The **Load** Stage outputs a series of SQL INSERT statements.

## Intro: Dependencies and Script Parameters

In [1]:
import requests
import re
from bs4 import BeautifulSoup

# If the VERBOSE flag is set, then intermediate steps in the data pipeline will be shown.
VERBOSE = True

# This is the input - a list of imdb IDs for the films that will compose the database.
imdb_ids  =  ['tt0859635', 'tt0486551', 'tt0331953', 'tt0456554', 'tt1135525', 'tt0245686', 
              'tt0114694', 'tt0115697', 'tt0144557', 'tt0348333', 'tt0366551', #one invalid in this row
              'tt0069239', 'tt0077914', 'tt0455407', 'tt0166843', 'tt0095652', 'tt1134854', 
              'tt0088993', 'tt0082622', 'tt0077402', 'tt0848557', 'tt0067341', 'tt0418819', 
              'tt0119177', 'tt0253556', 'tt0119116', 'tt0448134', 'tt0361862', 'tt0118929', 
              'tt1182345', 'tt0118884', 'tt0462504', 'tt0381849', 'tt0433387', 'tt0114746']

## Intro: Helper functions

These (mostly pretty simple) functions help the ETL pipeline below, but are not directly related to its logic, so are placed here to help the later sections of code flow better.  There are no dependencies among the helper functions, so they are in alphabetical order.

In [2]:
# find_the_money( s)
# find_the_time( s)
#
# These functions search the text string s for a
# dollar or minute value and return the corresponding
# quantity as an integer
#
def find_the_money( s):
    dollar_amounts = re.findall(r'\$[0-9,]+', s)
    for d in dollar_amounts:
        return int(d[1:].replace(',', ''))
    return None
    
    
def find_the_time( s):
    minute_amounts = re.findall(r'[0-9]+ min', s)
    for m in minute_amounts:
        return int(m[:-4])
    return None

# print_dd(  name, dd )
# print_lol( name, lol)
# print_section_header( s)
#
# These functions nicely print out either a data dictionary,
# a list-of-lists, or a fancy section header,
# Used for VERBOSE mode

def print_dd( name, dd):
    print(f'{name}\n' + '-'*len(name))
    for k, v in dd.items():
        print( f'\t{k}\t:\t{v}') 

def print_lol( name, lol):
    print(f'{name}\n' + '-'*len(name))        
    for l in lol:
            print('\t'+str(l))       

def print_section_header( s):
    fancy_title  = f'||     {s}     ||'
    fancy_break = '||' + ' '*(len(fancy_title) - 4) + '||'
    fancy_line = '=' * len(fancy_title) 
    print( '\n'.join([
        fancy_line, 
        fancy_break,
        fancy_title, 
        fancy_break,
        fancy_line
    ]))

# search_in_soup
# Looks inside of soup for a BeautifulSoup object wih
# a base tag of tag_type containing search_text,
# and returns the first success.
# 
def search_in_soup(soup, tag_type, search_text):
    candidates = soup.find_all(tag_type)
    
    for c in candidates:
        tag_text = c.text
        if re.search(search_text, tag_text):
            return c
    return None

# sql_insert_from_dd
# Generates the SQL statement to load the data dictionary dd
# into a table named table_name.
#
def sql_insert_from_dd( table_name, dd):
    return_string = f'INSERT INTO {table_name}'
    k_s = []
    v_s = []
    for k, v in dd.items():
        k_s.append(k)
        if isinstance( v, str):
            v_s.append( f'"{v}"')
        elif v == None:
            v_s.append('Null')
        else:
            v_s.append(str(v))
    return f'{return_string} ({", ".join(k_s)}) VALUES ({", ".join(v_s)});'
 
# text_field_s
# text_field_m
# text_field_l
#
# Short, medium, and long text fields.
# Truncate to the appropriate length and
# replace all ' with "
#
def text_field(s, l):
    TEXT_LENGTH = l
    t = s.lstrip().replace( '"', "'")
    if len(t) > TEXT_LENGTH:
        t = t[:TEXT_LENGTH-3] + '...'
    return t 

def text_field_l(s):
    return text_field(s, 400)

def text_field_m(s):
    return text_field(s, 200)

def text_field_s(s):
    return text_field(s, 45)

# Extract
## Extract: the main metadata extraction script

In [3]:
def scrape_all_metadata(imdb_id):
    movie_dd = scrape_movie_info(imdb_id)
    
    all_lols = { # lol for list-of-lists
        "Tagline" : scrape_taglines(imdb_id), 
        "Quote"   : scrape_quotes(imdb_id),
        "Summary": scrape_summaries(imdb_id),
        
        "Production": scrape_production_info(imdb_id),
        "Genre": scrape_genres(imdb_id),
        "Person": scrape_cast(imdb_id),
    }
    cast_we_care_about = get_cast_names( imdb_id)
    all_lols["Award"] = scrape_awards( imdb_id, cast_we_care_about)
    
    return movie_dd, all_lols   

## Extract: information from the main IMDB page

In [4]:
def scrape_movie_info(imdb_id):
    imdb_url =  f"https://www.imdb.com/title/{imdb_id}/"
    r       =   requests.get(imdb_url)
    soup    =   BeautifulSoup(r.text, 'html.parser')
    title_div = soup.find('div', class_='title_wrapper')
    
    movie_dd = {}
    movie_dd['movie_id'] = imdb_id
    
    title_div = soup.find('div', class_='title_wrapper')
    title_and_year = title_div.h1.text

    try:
        movie_year = int(title_and_year[-6:-2])
    except:
        movie_year = None
    movie_dd['movie_year'] = movie_year
    
    movie_title = title_and_year[:-8]
    movie_dd['movie_title'] = text_field_m(movie_title)
    
    details_div = soup.find('div', id='titleDetails')
    try:
        budget_div  = search_in_soup( details_div, 'div', 'Budget')
        budget = find_the_money(budget_div.text)
    except:
        budget = None
    movie_dd['movie_budget']     = budget  
    
    try:
        boxOffice_div  = search_in_soup( details_div, 'div', 'Cumulative Worldwide Gross')
        boxOffice = find_the_money(boxOffice_div.text)
    except:
        boxOffice = None
    movie_dd['movie_boxoffice'] = boxOffice
    
    runtime_div = search_in_soup( details_div, 'div', 'Runtime')
    runtime = find_the_time( runtime_div.text)
    movie_dd['movie_runtime'] = runtime
    
    return movie_dd

In [5]:
def scrape_genres(imdb_id):
    imdb_url =  f"https://www.imdb.com/title/{imdb_id}/"
    r       =   requests.get(imdb_url)
    soup    =   BeautifulSoup(r.text, 'html.parser')
    
    storyline_div = soup.find('div', id='titleStoryLine')
    genres_div  = search_in_soup( storyline_div, 'div', 'Genre')
    genre_links = genres_div.find_all('a')

    return_list = []
    for l in genre_links:
        genre_item = [imdb_id, text_field_s(l.text)]
        if genre_item not in return_list:
            return_list.append(genre_item)

    return return_list

## Extract: a single text field from a single page

In [6]:
def scrape_taglines(imdb_id):

    # web setup
    full_link = 'https://www.imdb.com/title/' + imdb_id + '/taglines?ref_=tt_stry_tg'

    source = requests.get(full_link).text
    soup = BeautifulSoup(source, 'html.parser')

    # list of all attributes
    soda_lst = soup.find_all('div', class_='soda odd')

    # tagline
    tagline_lst = list(soda_lst)
    real_lst = []

    # creating lists
    for line in range(2):
        try:
            real_lst.append([imdb_id, text_field_m(tagline_lst[line].text.strip())])
        except:
            pass

    return real_lst


def scrape_quotes(imdb_id):

    # web setup
    full_link = 'https://www.imdb.com/title/' + imdb_id + '/quotes/?tab=qt&ref_=tt_trv_qu'

    source = requests.get(full_link).text
    soup = BeautifulSoup(source, 'html.parser')

    # list of all attributes
    l_lst = soup.find('div', class_='list')

    # quotes
    quote_lst = list(l_lst)
    real_quote = []

    for quote in range(4):
        try:
            real_quote.append([imdb_id, text_field_l(quote_lst[quote].find('div', class_='sodatext').text)])
        except:
            pass

    return real_quote


def scrape_summaries(imdb_id):

    # web setup
    full_link = 'https://www.imdb.com/title/' + imdb_id + '/plotsummary?ref_=tt_stry_pl'

    source = requests.get(full_link).text
    soup = BeautifulSoup(source, 'html.parser')

    # list of attributes
    zebra = soup.find('ul', class_='ipl-zebra-list')
    zebra_lst = zebra.find_all('li', class_='ipl-zebra-list__item')

    # summaries
    sum_lst = list(zebra_lst)
    real_lst = []

    for line in range(2):
        try:
            real_lst.append([imdb_id, text_field_l(sum_lst[line].text.strip())])
        except:
            pass

    return real_lst


def scrape_production_info(imdb_id):
    # web setup
    full_link = 'https://www.imdb.com/title/' + imdb_id + '/companycredits?ref_=ttfc_ql_4'

    source = requests.get(full_link).text
    soup = BeautifulSoup(source, 'html.parser')

    # list with all attributes
    simple_lst = soup.find_all('ul', class_='simpleList')

    # production companies
    production_lst = list(simple_lst[0])
    real_lst = []

    # creating lists
    for tag in range(len(production_lst)):
        try:
            prod_item = [imdb_id, text_field_s(production_lst[tag].a.text)]
            if prod_item not in real_lst:
                real_lst.append(prod_item)
        except:
            pass

    return real_lst


# Extract: cast and awards

In [7]:
def scrape_cast(imdb_id):
    url = "https://www.imdb.com/title/" + "imdb_id"
    casts = []
    names = get_cast_names(imdb_id)
    check = 0
    stop = len(names)
    while check < stop:
        if check == 0 or check == 1:
            casts.append([imdb_id, check, text_field_s(names[check])])
            check += 1
        else:
            casts.append([imdb_id, 2, text_field_s(names[check])])
            check += 1
    return casts

def get_cast_names(imdb_id):
    url = "https://www.imdb.com/title/" + imdb_id
    names = []
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    out = soup.find_all('div', class_='credit_summary_item')
    directors = out[0].text.split()
    director = directors[1] + ' ' + directors[2]
    names.append(text_field_s(director))
    writers = out[1].text.split()
    writer = writers[1] + ' ' + writers[2]
    names.append(text_field_s(writer))
    stars = out[2].text.split('|')[0].split('\n')[2].split(',')
    for i in stars:
        if i not in names:
            names.append(text_field_s(i))
    return names

def scrape_awards(imdb_id, important):
    results = []
    awd_url = "https://www.imdb.com/title/" + imdb_id + "/awards"
    awd_r = requests.get(awd_url)
    awd_soup = BeautifulSoup(awd_r.text, 'html.parser')
  
    awd_out = awd_soup.find_all('table', class_='awards')
    #print(len(awd_out))
    awards = {}
    for block in awd_out:
        text = block.text.split(' '*6)
        text = [i for i in text if i !='']
        #print(text)
        prefix = [i for i in text[0].split('\n') if i != '']
        prefix = ' '.join([str(elem) for elem in prefix[::-1]])
        for i in text[1:]:
            record = [j for j in i.split('\n') if j != '']
            if len(record) > 0:
                suffix = record[0]
                award = prefix + ': ' + suffix
                people = tuple(record[1:])
                awards[people] = award
    #print(awards)
    results = []
    for p in important:
        if len(results) == 3:
            return results
        for k,v in awards.items():
            if p in k:
                results.append([imdb_id, p, text_field_m(v)])
                break
    
    if len(results) < 1:
        for k, v in awards.items():
            if k not in important:
                results.append([imdb_id, None, text_field_m(v)])
    
    return results

In [8]:
imdb_id = 'tt0859635'
cast_we_care_about = get_cast_names( imdb_id)

print(cast_we_care_about)
print( scrape_awards(imdb_id, cast_we_care_about ))

['Jay Chandrasekhar', 'Jay Chandrasekhar', 'Kevin Heffernan', 'Steve Lemme ']
[['tt0859635', None, 'Golden Trailer Nominee: Best Radio/Audio Spot'], ['tt0859635', None, 'Golden Trailer Winner: Best Teaser Poster'], ['tt0859635', None, 'Golden Trailer Winner: Best Trailerbyte for a Feature Film']]


## Extract: extraction loop

In [9]:
ready_to_load = {}
ready_to_load['Movie'] = []

raw_data = {
    "Tagline"   : [],
    "Quote"   : [],
    "Summary": [],
    "Production": [],
    "Genre": [],
    "Person": [],    
    "Award": []
}

for id in imdb_ids:
    print(f'Scraping from {id}...')
    movie_dd, lols = scrape_all_metadata(id)
    
    ready_to_load['Movie'].append(movie_dd)
    
    for name, lol in lols.items():
        try:
            raw_data[name].extend(lol) 
        except:
            pass

Scraping from tt0859635...
Scraping from tt0486551...
Scraping from tt0331953...
Scraping from tt0456554...
Scraping from tt1135525...
Scraping from tt0245686...
Scraping from tt0114694...
Scraping from tt0115697...
Scraping from tt0144557...
Scraping from tt0348333...
Scraping from tt0366551...
Scraping from tt0069239...
Scraping from tt0077914...
Scraping from tt0455407...
Scraping from tt0166843...
Scraping from tt0095652...
Scraping from tt1134854...
Scraping from tt0088993...
Scraping from tt0082622...
Scraping from tt0077402...
Scraping from tt0848557...
Scraping from tt0067341...
Scraping from tt0418819...
Scraping from tt0119177...
Scraping from tt0253556...
Scraping from tt0119116...
Scraping from tt0448134...
Scraping from tt0361862...
Scraping from tt0118929...
Scraping from tt1182345...
Scraping from tt0118884...
Scraping from tt0462504...
Scraping from tt0381849...
Scraping from tt0433387...
Scraping from tt0114746...


In [10]:
if VERBOSE:
    print( "There is no list-of-lists for the Movie table; that data is already in the finalized data dict.\n")
    for name, lol in raw_data.items():
        print_lol(name, lol)

There is no list-of-lists for the Movie table; that data is already in the finalized data dict.

Tagline
-------
	['tt0859635', 'The time is meow.']
	['tt0859635', "Don't Miss It!"]
	['tt0486551', "Bring on the beer. They've got the nuts."]
	['tt0486551', 'Comedy on tap']
	['tt0331953', 'A vacation to die for.']
	['tt0331953', 'Sun, Sea, Sex and Slaughter (Ireland DVD)']
	['tt0456554', 'Sex. Drugs. Nakedness. Rude language... And proud of it!']
	['tt1135525', "Bring a bib, it's gonna get messy"]
	['tt0245686', "Keep on Moppin' in the Free world."]
	['tt0245686', 'He Came. He Cleaned. He Conquered']
	['tt0114694', "If at first you don't succeed, lower your standards."]
	['tt0115697', "There's one in every family."]
	['tt0348333', 'Hungry for Laughs? (DVD)']
	['tt0348333', 'A comedy of massive portions']
	['tt0366551', 'Fast Food. High Times.']
	['tt0069239', 'Caviar in the kitchen - nothing in the bedroom']
	['tt0069239', "first NIGHT OF THE LIVING DEAD then DAWN OF THE DEAD - NOW Georg

# Transform
## Transform: only needs an identifier

In [11]:
# Taglines, Quotes, and Summaries only need a unique identifier attached to each value.
needs_an_identifier = [
    'Tagline',
    'Summary',
    'Quote'
]
for t_name in needs_an_identifier:   
    table_data = []
    for i, [movie_id, text] in enumerate( raw_data[t_name]):   
        dd = {
            f'{t_name.lower()}_id': i,
            'movie_id': movie_id,
            f'{t_name.lower()}_text': text
        }
        table_data.append(dd)
    ready_to_load[t_name] = table_data

## Transform: implement M:N relationship

In [12]:
MN_relationship = {
    # a:b where
    #  a is key for the raw, unnormalized data
    #  b is the name for the new intermediate table
    #
    'Genre' : 'MovieGenre',
    'Production' : 'MovieProduction',
    'Person': 'Role'
}
MN_cols = {
    'Genre' : ['movie_id', 'genre_id'],
    'Production': ['movie_id', 'production_id'],
    'Person': ['movie_id', 'role_code', 'person_id']
}

for t_o, t_MN in MN_relationship.items():
    unnormalized_data = raw_data[ t_o]
    connections = []
    
    final_column_values = set()
    for r in unnormalized_data:
        final_column_values.add(r[-1])
 
    value_ids = []
    unique_col_ids = {}
    for col_id, col_text in enumerate(final_column_values):
        unique_col_ids[col_id] = col_text
        value_ids.append({
            f'{t_o.lower()}_id' : col_id,
            f'{t_o.lower()}_name' : col_text
        })
    for r in unnormalized_data:
        dd = {}
        current_final_col = r[-1]
        
        for col_id, col_text in unique_col_ids.items():
            if col_text == current_final_col:
                MN_dd = {}
                for i in range( len(r) - 1 ):
                    MN_dd[MN_cols[t_o][i]] = r[i]
                MN_dd[MN_cols[t_o][-1]] = col_id
                connections.append( MN_dd)
        
    ready_to_load[ t_o] = value_ids
    ready_to_load[t_MN] = connections
        

## Transform: match awards to person

In [13]:
awards = []

i = 0
for r in raw_data['Award']:
    person_id = None
    for entry in ready_to_load['Person']:
        if entry['person_name'] == r[1]:
            person_id = entry['person_id']

    awards.append({
        'award_id' : i,
        'movie_id' : r[0],
        'person_id': person_id,
        'award_text': r[2]
    })
    i = i + 1
ready_to_load['Award'] = awards

## Transform: Rolecodes are hardcoded

In [14]:
ready_to_load['RoleCode'] = [
    {
        'role_code': 0, 
        'role_name': 'Director'
    },{
        'role_code': 1, 
        'role_name': 'Writer'
    },{
        'role_code': 2, 
        'role_name': 'Actor'
    }
]

## Transform: show finalized data if VERBOSE

In [15]:
if VERBOSE:
    table_num = 0
    for t_name, t_data in ready_to_load.items():
        table_num = table_num + 1
        print_section_header(f'{table_num}. {t_name}')

        num_rows = len( t_data)
        for i in range( num_rows):
            print_dd(f'{t_name} {str(i)}', t_data[i])
        print()

||                  ||
||     1. Movie     ||
||                  ||
Movie 0
-------
	movie_id	:	tt0859635
	movie_year	:	2018
	movie_title	:	Super Troopers 2
	movie_budget	:	13500000
	movie_boxoffice	:	31626386
	movie_runtime	:	100
Movie 1
-------
	movie_id	:	tt0486551
	movie_year	:	2006
	movie_title	:	Beerfest
	movie_budget	:	17500000
	movie_boxoffice	:	20387597
	movie_runtime	:	110
Movie 2
-------
	movie_id	:	tt0331953
	movie_year	:	2004
	movie_title	:	Club Dread
	movie_budget	:	8600000
	movie_boxoffice	:	7565807
	movie_runtime	:	104
Movie 3
-------
	movie_id	:	tt0456554
	movie_year	:	2006
	movie_title	:	Grandma's Boy
	movie_budget	:	None
	movie_boxoffice	:	6566277
	movie_runtime	:	94
Movie 4
-------
	movie_id	:	tt1135525
	movie_year	:	2009
	movie_title	:	The Slammin' Salmon
	movie_budget	:	None
	movie_boxoffice	:	60421
	movie_runtime	:	99
Movie 5
-------
	movie_id	:	tt0245686
	movie_year	:	2001
	movie_title	:	Joe Dirt
	movie_budget	:	17700000
	movie_boxoffice	:	30987695
	movie_runti

	production_id	:	2
	production_name	:	Votiv Films
Production 3
------------
	production_id	:	3
	production_name	:	UK Film Council
Production 4
------------
	production_id	:	4
	production_name	:	Moving Picture Company (MPC)
Production 5
------------
	production_id	:	5
	production_name	:	Cataland Films
Production 6
------------
	production_id	:	6
	production_name	:	Classico
Production 7
------------
	production_id	:	7
	production_name	:	Participant
Production 8
------------
	production_id	:	8
	production_name	:	Overture Films
Production 9
------------
	production_id	:	9
	production_name	:	Liberty Films UK
Production 10
-------------
	production_id	:	10
	production_name	:	DNA Films
Production 11
-------------
	production_id	:	11
	production_name	:	Kingsgate Films
Production 12
-------------
	production_id	:	12
	production_name	:	The Latent Image
Production 13
-------------
	production_id	:	13
	production_name	:	Universal Pictures
Production 14
-------------
	production_id	:	14
	production

	person_name	:	Christine Forrest 
Person 38
---------
	person_id	:	38
	person_name	:	Jeremy Davies 
Person 39
---------
	person_id	:	39
	person_name	:	Joseph Pilato 
Person 40
---------
	person_id	:	40
	person_name	:	Bill Paxton
Person 41
---------
	person_id	:	41
	person_name	:	Robert Zemeckis
Person 42
---------
	person_id	:	42
	person_name	:	Izabella Scorupco 
Person 43
---------
	person_id	:	43
	person_name	:	Shirley Jones 
Person 44
---------
	person_id	:	44
	person_name	:	Milla Jovovich
Person 45
---------
	person_id	:	45
	person_name	:	Scott H. Reiniger 
Person 46
---------
	person_id	:	46
	person_name	:	Werner Herzog
Person 47
---------
	person_id	:	47
	person_name	:	Halsted Welles
Person 48
---------
	person_id	:	48
	person_name	:	Danny Leiner
Person 49
---------
	person_id	:	49
	person_name	:	Rudy Ricci
Person 50
---------
	person_id	:	50
	person_name	:	Lincoln Maazel
Person 51
---------
	person_id	:	51
	person_name	:	George A.
Person 52
---------
	person_id	:	52
	person_name

-------
	movie_id	:	tt1134854
	role_code	:	2
	person_id	:	93
Role 81
-------
	movie_id	:	tt0088993
	role_code	:	0
	person_id	:	51
Role 82
-------
	movie_id	:	tt0088993
	role_code	:	1
	person_id	:	51
Role 83
-------
	movie_id	:	tt0088993
	role_code	:	2
	person_id	:	55
Role 84
-------
	movie_id	:	tt0088993
	role_code	:	2
	person_id	:	110
Role 85
-------
	movie_id	:	tt0088993
	role_code	:	2
	person_id	:	39
Role 86
-------
	movie_id	:	tt0082622
	role_code	:	0
	person_id	:	51
Role 87
-------
	movie_id	:	tt0082622
	role_code	:	1
	person_id	:	51
Role 88
-------
	movie_id	:	tt0082622
	role_code	:	2
	person_id	:	58
Role 89
-------
	movie_id	:	tt0082622
	role_code	:	2
	person_id	:	81
Role 90
-------
	movie_id	:	tt0082622
	role_code	:	2
	person_id	:	85
Role 91
-------
	movie_id	:	tt0077402
	role_code	:	0
	person_id	:	51
Role 92
-------
	movie_id	:	tt0077402
	role_code	:	1
	person_id	:	51
Role 93
-------
	movie_id	:	tt0077402
	role_code	:	2
	person_id	:	14
Role 94
-------
	movie_id	:	tt0077402
	ro

	person_id	:	65
	award_text	:	Hugo Nominee: Best Dramatic Presentation
Award 55
--------
	award_id	:	55
	movie_id	:	tt0114746
	person_id	:	21
	award_text	:	Saturn Award Winner: Best Actor

||                      ||
||     12. RoleCode     ||
||                      ||
RoleCode 0
----------
	role_code	:	0
	role_name	:	Director
RoleCode 1
----------
	role_code	:	1
	role_name	:	Writer
RoleCode 2
----------
	role_code	:	2
	role_name	:	Actor



# Load: generate INSERT INTO statements

In [16]:
x

INSERT INTO Movie (movie_id, movie_year, movie_title, movie_budget, movie_boxoffice, movie_runtime) VALUES ("tt0859635", 2018, "Super Troopers 2", 13500000, 31626386, 100);
INSERT INTO Movie (movie_id, movie_year, movie_title, movie_budget, movie_boxoffice, movie_runtime) VALUES ("tt0486551", 2006, "Beerfest", 17500000, 20387597, 110);
INSERT INTO Movie (movie_id, movie_year, movie_title, movie_budget, movie_boxoffice, movie_runtime) VALUES ("tt0331953", 2004, "Club Dread", 8600000, 7565807, 104);
INSERT INTO Movie (movie_id, movie_year, movie_title, movie_budget, movie_boxoffice, movie_runtime) VALUES ("tt0456554", 2006, "Grandma's Boy", Null, 6566277, 94);
INSERT INTO Movie (movie_id, movie_year, movie_title, movie_budget, movie_boxoffice, movie_runtime) VALUES ("tt1135525", 2009, "The Slammin' Salmon", Null, 60421, 99);
INSERT INTO Movie (movie_id, movie_year, movie_title, movie_budget, movie_boxoffice, movie_runtime) VALUES ("tt0245686", 2001, "Joe Dirt", 17700000, 30987695, 91);
I

INSERT INTO Person (person_id, person_name) VALUES (80, "Brad Anderson");
INSERT INTO Person (person_id, person_name) VALUES (81, "Gary Lahti");
INSERT INTO Person (person_id, person_name) VALUES (82, "Zohra Lampert");
INSERT INTO Person (person_id, person_name) VALUES (83, "Judith Ridley");
INSERT INTO Person (person_id, person_name) VALUES (84, "Radha Mitchell");
INSERT INTO Person (person_id, person_name) VALUES (85, "Tom Savini ");
INSERT INTO Person (person_id, person_name) VALUES (86, "David Spade,");
INSERT INTO Person (person_id, person_name) VALUES (87, "Kate McNeil ");
INSERT INTO Person (person_id, person_name) VALUES (88, "Rufus Sewell");
INSERT INTO Person (person_id, person_name) VALUES (89, "Linda Cardellini");
INSERT INTO Person (person_id, person_name) VALUES (90, "Gary Oldman ");
INSERT INTO Person (person_id, person_name) VALUES (91, "Kevin O'Connor ");
INSERT INTO Person (person_id, person_name) VALUES (92, "Gregg Chabot");
INSERT INTO Person (person_id, person_name

INSERT INTO Role (movie_id, role_code, person_id) VALUES ("tt0095652", 2, 62);
INSERT INTO Role (movie_id, role_code, person_id) VALUES ("tt0095652", 2, 87);
INSERT INTO Role (movie_id, role_code, person_id) VALUES ("tt1134854", 0, 51);
INSERT INTO Role (movie_id, role_code, person_id) VALUES ("tt1134854", 1, 51);
INSERT INTO Role (movie_id, role_code, person_id) VALUES ("tt1134854", 2, 24);
INSERT INTO Role (movie_id, role_code, person_id) VALUES ("tt1134854", 2, 96);
INSERT INTO Role (movie_id, role_code, person_id) VALUES ("tt1134854", 2, 93);
INSERT INTO Role (movie_id, role_code, person_id) VALUES ("tt0088993", 0, 51);
INSERT INTO Role (movie_id, role_code, person_id) VALUES ("tt0088993", 1, 51);
INSERT INTO Role (movie_id, role_code, person_id) VALUES ("tt0088993", 2, 55);
INSERT INTO Role (movie_id, role_code, person_id) VALUES ("tt0088993", 2, 110);
INSERT INTO Role (movie_id, role_code, person_id) VALUES ("tt0088993", 2, 39);
INSERT INTO Role (movie_id, role_code, person_id) V